In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import wrds
import datetime
import statsmodels.api as sm

In [2]:
db = wrds.Connection() 
#goqhuB-1hafqe-dojvix

Enter your WRDS username [max]:antb95
Enter your password:········
WRDS recommends setting up a .pgpass file.
You can find more info here:
https://www.postgresql.org/docs/9.5/static/libpq-pgpass.html.
Loading library list...
Done


In [3]:
sp = db.get_table(library = 'crsp' , table = 'dsp500p')
start = np.where(sp.caldt >= datetime.date(2011,3,17))[0][0]
end = np.where(sp.caldt >= datetime.date(2016,3,17))[0][0]
sp = sp.loc[start:end,['caldt','spindx']]

In [4]:
names = db.get_table(library='crsp', table='stocknames')
names =names.loc[:,['permco' , 'ticker']]

In [5]:
Ticker = ['IBM','MCD','MMM','WMT']
Permco = np.zeros(len(Ticker))
for i in range(len(Ticker)):
    inter = names['permco'].where(names['ticker'] == Ticker[i])
    inter.dropna(inplace = True)
    Permco[i] = inter.iloc[1]

In [6]:
df = pd.DataFrame(index = np.arange(0,len(sp)),columns = ['Date' , 'SPX'] + Ticker)

In [7]:
df.Date = np.array(sp.caldt)
df.SPX = np.array(sp.spindx)
for i in range(len(Ticker)):
    inter = db.raw_sql("select prc, date from crsp.dsf where permco in ("+str(Permco[i])+") and date >='2011-03-17' and date <='2016-03-16'")
    df[Ticker[i]] = inter.prc

In [8]:
df.loc[:,df.columns != 'Date'] = np.log(df.loc[:,df.columns != 'Date']) - np.log(df.loc[:,df.columns != 'Date']).shift(1)
df.dropna(inplace = True)

In [9]:
X = pd.DataFrame({'Ones' : [1] * len(df), 'SPX' : np.array(df.SPX)},index = np.arange(1,len(df)+1,1))
Beta = pd.DataFrame(columns = Ticker,index = ['a' , 'Beta'])
Eps = pd.DataFrame(columns = Ticker, index = df.index)
for i in range(len(Ticker)):
    Y = df[Ticker[i]]
    model = sm.OLS(Y,X).fit()
    Beta.at[:,Ticker[i]] = model.params.to_list()
    Eps.at[:,Ticker[i]] = Y - np.matmul(np.array(X),np.array(Beta[Ticker[i]]))
    print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                    IBM   R-squared:                       0.420
Model:                            OLS   Adj. R-squared:                  0.419
Method:                 Least Squares   F-statistic:                     908.1
Date:                Mon, 21 Oct 2019   Prob (F-statistic):          1.50e-150
Time:                        22:39:57   Log-Likelihood:                 4072.4
No. Observations:                1257   AIC:                            -8141.
Df Residuals:                    1255   BIC:                            -8131.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Ones          -0.0004      0.000     -1.308      0.1

,IBM,MCD,MMM,WMT
a,-0.000350,0.000198,0.000122,0.000030
Beta,0.812348,0.583537,0.977800,0.522851


In [10]:
Beta

,IBM,MCD,MMM,WMT
a,-0.000350,0.000198,0.000122,0.000030
Beta,0.812348,0.583537,0.977800,0.522851


In [11]:
Eps.corr()

,IBM,MCD,MMM,WMT
IBM,1.000000,0.063572,0.001717,-0.002494
MCD,0.063572,1.000000,0.023304,0.121076
MMM,0.001717,0.023304,1.000000,0.029484
WMT,-0.002494,0.121076,0.029484,1.000000


In [12]:
df[Ticker].corr()

,IBM,MCD,MMM,WMT
IBM,1.000000,0.426359,0.528267,0.320711
MCD,0.426359,1.000000,0.497808,0.381751
MMM,0.528267,0.497808,1.000000,0.419896
WMT,0.320711,0.381751,0.419896,1.000000
